# Case study - 4TT - introducing the DOAS and the IRFAS


Recently, it has been demonstrated that coherent vibrational modes promote the ultrafast internal conversion and intersystem crossing in thiobases. The global and target analysis of 4-thiothymidine (4TT) in Phosphate-Buffered Saline (PBS) has been described in
Teles-Ferreira DC, van Stokkum IHM, Conti I, Ganzer L, Manzoni C, Garavelli M, Cerullo G, Nenov A, Borrego-Varillas R, de Paula AM (2022) Coherent vibrational modes promote the ultrafast internal conversion and intersystem crossing in thiobases. Physical Chemistry Chemical Physics 24 (36):21750-21758. doi:10.1039/D2CP02073D


![figure4TT](./4TT.jpg)

The structure of 4TT (inset) with its normalized absorption spectrum (black curve), the normalized pump pulse spectrum (red curve), and normalized photoluminescence spectrum (blue curve is the fit and blue dots are the data) obtained pumping the sample at 330 nm. Figure adopted from Teles-Ferreira et al. (2022)


## Define and inspect data


In [ ]:
from pyglotaran_extras import plot_data_overview

experiment_data = "experiment_data/4TT_PBS.ascii"

plot_data_overview(experiment_data, linlog=True, linthresh=1);

# Create a project and import the data

In [ ]:
from glotaran.project import Project

project = Project.open("")
project.import_data(experiment_data, dataset_name="4TT")

## Model and parameter definitions


In [ ]:
project.show_model_definition("sequential")

In [ ]:
project.show_parameters_definition("sequential")

# Optimization


In [ ]:
result = project.optimize(
    model_name="sequential",
    parameters_name="sequential",
    maximum_number_function_evaluations=17,
)

## Inspect fit quality


In [ ]:
result

In [ ]:
result.optimized_parameters

In [ ]:
result.data["4TT"].lifetime_decay

## Populations and SADS estimated with the sequential scheme

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas


def plot_concentration_and_spectra(result_dataset):
    # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
    plot_concentrations(result_dataset, axes[0], center_λ=0, linlog=True)
    plot_sas(result_dataset, axes[1])
    return fig, axes


fig, axes = plot_concentration_and_spectra(result.data["4TT"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

## Plot fitted traces


In [ ]:
from pyglotaran_extras.plotting.plot_traces import plot_fitted_traces
from pyglotaran_extras.plotting.plot_traces import select_plot_wavelengths
from pyglotaran_extras.plotting.style import PlotStyle

wavelengths = select_plot_wavelengths(
    result.data["4TT"], equidistant_wavelengths=False, axes_shape=(4, 3)
)
fig3tr, axes = plot_fitted_traces(
    result,
    wavelengths,
    axes_shape=(4, 3),
    linlog=True,
    linthresh=1,
    cycler=PlotStyle().data_cycler_solid_dashed,
)
for ax in axes.flatten():
    ax_title = ax.get_title()
    ax.set_title(rf"{ax_title.replace('spectral', 'Wavelength')}$\,$nm")
    ax.set_xlabel("Time (ps)")

In [ ]:
from cycler import cycler

wavelengths = [406, 445, 510, 601]
fig3tr, axes = plot_fitted_traces(
    result,
    wavelengths,
    axes_shape=(2, 2),
    linlog=True,
    linthresh=1,
    cycler=cycler(color=["r", "k"]),  # change plot colors
    # figsize=(10, 5),
    figsize=(8, 4),
)

# Add thin line at zero to all plots
for ax in axes.flatten():
    ax.axhline(0, color="k", linewidth=1)
    ax.set_xlabel("Time (ps)")
    ax.set_ylabel("")
    ax_title = ax.get_title()
    ax.set_title(rf"{ax_title.replace('spectral', 'Wavelength')}$\,$nm")

In [ ]:
wavelengths = [406, 601]
fig3tr, axes = plot_fitted_traces(
    result,
    wavelengths,
    axes_shape=(1, 2),
    linlog=True,
    linthresh=1,
    cycler=cycler(color=["r", "k"]),  # change plot colors
    # figsize=(10, 5),
    figsize=(8, 3),
)

# Add thin line at zero to all plots
for ax in axes.flatten():
    ax.axhline(0, color="k", linewidth=1)
    ax.set_xlabel("Time (ps)")
    ax.set_ylabel("")
    ax_title = ax.get_title()
    ax.set_title(rf"{ax_title.replace('spectral', 'Wavelength')}$\,$nm")

## Overview


In [ ]:
from pyglotaran_extras import plot_overview

fig, axes = plot_overview(result, linlog=True, linthresh=1, nr_of_residual_svd_vectors=1)

# Add thin line at zero to SAS and DAS plots
for ax in axes[0:2, 1:3].flatten():
    ax.axhline(0, color="k", linewidth=1)

## Overview of the estimated DOAS and phases of the uncorrected data

In [ ]:
from pyglotaran_extras import plot_doas

fig, axes = plot_doas(
    result,
    damped_oscillation=["osc1", "osc2", "osc3"],
    time_range=(-0.05, 0.3),
    spectral=550,
    figsize=(15, 4),
    # oscillation_type="sin",
    # normalize=False,
)

# for vline_pos in [412,450]:
#     axes[1].axvline(vline_pos, color="k", linewidth=1)
#     axes[2].axvline(vline_pos, color="k", linewidth=1)
axes[0].set_xlabel("Time (ps)")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[1].set_title("DOAS")
axes[0].annotate("A", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)

## Plot coherent artifact


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

irfas_plot_wavelength = 550

fig, axes = plot_coherent_artifact(
    result.data["4TT"], time_range=(-0.1, 0.1), spectral=irfas_plot_wavelength, figsize=(12, 3.5)
)
axes[0].set_xlabel("Time (ps)")
# axes[0].get_legend().remove()
axes[1].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("")
axes[0].annotate("A", xy=(0.01, 0.9), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.01, 0.9), xycoords="axes fraction", fontsize=16)
fig.tight_layout()

## Residual analysis of the uncorrected data

In [ ]:
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


def plot_residual_and_svd(result_dataset):
    fig, axes = plt.subplots(1, 3, figsize=(10, 2))
    plot_residual(result_dataset, axes[0])
    axes[0].get_legend().remove()
    axes[0].set_ylabel("Wavelength (nm)")
    plot_lsv_residual(result_dataset, axes[1], indices=[0])
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")
    axes[1].set_title("residual 1st LSV")
    plot_rsv_residual(result_dataset, axes[2], indices=[0])
    axes[2].set_xlabel("Wavelength (nm)")
    axes[2].set_title("residual 1st RSV")
    axes[2].get_legend().remove()
    axes[2].set_ylabel("")

    return fig, axes


fig, axes = plot_residual_and_svd(result.data["4TT"])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

## Refinement by estimation of the laser intensity fluctuations responsible for the residual structure


In [ ]:
import numpy as np
import xarray as xr

alfa = []
for timepoint in result.data["4TT"].time:
    y = result.data["4TT"].residual.sel(time=timepoint)
    x = result.data["4TT"].data.sel(time=timepoint)
    alfa.append(np.dot(y, x) / np.dot(x, x))

xr.DataArray(alfa).plot()

In [ ]:
alfa = []
talfa = []
for timepoint in result.data["4TT"].time:
    y = result.data["4TT"].residual.sel(time=timepoint)
    x = result.data["4TT"].fitted_data.sel(time=timepoint)
    xtx = np.dot(x, x)
    xty = np.dot(x, y)
    a = xty / xtx
    res = y - a * x
    df = len(y) - 1
    var = np.dot(res, res) / df
    ta = a / np.sqrt(var / xtx)
    alfa.append(a)
    talfa.append(ta)

talfa_xr = xr.DataArray(talfa, coords={"time": result.data["4TT"].time})
alfa_xr = xr.DataArray(alfa, coords={"time": result.data["4TT"].time})
alfaraw_xr = alfa_xr
alfa_xr[alfa_xr.time < -0.08] = 0
alfa_xr.plot()
talfa_xr.plot()

In [ ]:
fig, ax = plt.subplots(1, 1)
alfa_xr.plot()
ax.set_xlabel("Time (ps)")
ax.set_ylabel("ΔI")

## Correcting the data for the laser intensity fluctuations


In [ ]:
fig, ax = plt.subplots(1, 1)
corr4TT = result.data["4TT"].data / (1 + alfa_xr)
corr4TT.plot(x="time", y="spectral")
ax.set_ylabel("Wavelength (nm)")
ax.set_xlabel("Time (ps)")

## Target analysis of the corrected data


In [ ]:
project.import_data(corr4TT, dataset_name="corr4TT")
plot_data_overview(project.data["corr4TT"], linlog=True, linthresh=1)

In [ ]:
result_corr = project.optimize(
    model_name="sequential6osc",
    parameters_name="optimized_parameters_seq4osc6",
    maximum_number_function_evaluations=7,
)
result_corr

In [ ]:
from pyglotaran_extras import plot_overview

fig, axes = plot_overview(result_corr, linlog=True, linthresh=1)

# Add thin line at zero to SAS and DAS plots
for ax in axes[0:2, 1:3].flatten():
    ax.axhline(0, color="k", linewidth=1)

In [ ]:
wavelengths = [406, 445, 510, 601]
fig3tr, axes = plot_fitted_traces(
    result_corr,
    wavelengths,
    axes_shape=(2, 2),
    linlog=True,
    linthresh=0.3,
    cycler=cycler(color=["r", "k"]),  # change plot colors
    figsize=(10, 5),
)

# Add thin line at zero to all plots
for ax in axes.flatten():
    ax.axhline(0, color="k", linewidth=1)
    ax_title = ax.get_title()
    ax.set_title(rf"{ax_title.replace('spectral', 'Wavelength')}$\,$nm")
    ax.set_xlabel("Time (ps)")

In [ ]:
result_corr.optimized_parameters

Note that DOAS #3,4 have decay rates larger than 50, and most likely belong to the CA.
#1,5 with decay rates of 26,16 are in between, and #2 and 6 with decay rates of 9.6 and 2.6 are the longer lived damped oscillations.


In [ ]:
result_corr

# Plot coherent artifact


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

irfas_plot_wavelength = 550

plot_coherent_artifact(
    result_corr.data["corr4TT"],
    time_range=(-0.1, 0.1),
    spectral=irfas_plot_wavelength,
)

In [ ]:
result_corr.optimized_parameters

### Refining the kinetic model


Because the red SADS above still contains stimulated emission around 430 nm, a relaxed S2 is introduced, which is populated by 40% of the decaying S2,
whereas 60% of S2 converts to S1. With some educated guesses, with the help of spectral constraints, and after some trial and error,
reasonable starting values for the additional rate constants have been found. The thus estimated SADS can be interpreted.


In [ ]:
result_corr_refined = project.optimize(
    model_name="refined6osc",
    parameters_name="optimized_parameters_6osc_target",
    maximum_number_function_evaluations=1,
)
result_corr_refined

In [ ]:
result_corr_refined.data["corr4TT"].lifetime_decay

## Populations and SADS estimated with the refined kinetic scheme

In [ ]:
def plot_concentration_and_spectra(result_dataset):
    # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
    plot_concentrations(result_dataset, axes[0], center_λ=0, linlog=True)
    plot_sas(result_dataset, axes[1])
    return fig, axes


fig, axes = plot_concentration_and_spectra(result_corr_refined.data["corr4TT"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

In [ ]:
result_corr_refined.optimized_parameters

The damping rates 5 and 6 are 70.3/ps and -51.7/ps. These damped oscillations are attributable to the CA. 

## Decomposition of the 4TT in PBS data


In [ ]:
matrix = result_corr_refined.data["corr4TT"].matrix
clp = result_corr_refined.data["corr4TT"].clp


ca_labels = []
ca_doas_labels = []
doas_labels = []
non_doas_labels = []


for clp_label in matrix.clp_label:
    if clp_label.item().startswith("coherent_artifact_"):
        ca_labels.append(clp_label.item())
    elif clp_label.item().startswith(("osc5_", "osc6_")):
        ca_doas_labels.append(clp_label.item())
    elif clp_label.item().endswith(("_sin", "_cos")):
        doas_labels.append(clp_label.item())
    else:
        non_doas_labels.append(clp_label.item())

print(f"{doas_labels=}")
print(f"{ca_labels=}")
print(f"{ca_doas_labels=}")
print(f"{non_doas_labels=}")
# matrix.clp_label

In [ ]:
from pyglotaran_extras.plotting.utils import extract_irf_location
from pyglotaran_extras.plotting.utils import shift_time_axis_by_irf_location

data_dict = {
    "data": result_corr_refined.data["corr4TT"].data,
    "fitted_data": result_corr_refined.data["corr4TT"].fitted_data,
    # "sum": (matrix * clp).sum(dim="clp_label"),
}

for non_doas_label in non_doas_labels:
    data_dict[non_doas_label] = (
        matrix.sel(clp_label=non_doas_label) * clp.sel(clp_label=non_doas_label)
    ).drop("clp_label")

data_dict["doas"] = ((matrix.sel(clp_label=doas_labels) - 1) * clp.sel(clp_label=doas_labels)).sum(
    dim="clp_label"
)

data_dict["CA"] = (matrix.sel(clp_label=ca_labels) * clp.sel(clp_label=ca_labels)).sum(
    dim="clp_label"
) + ((matrix.sel(clp_label=ca_doas_labels) - 1) * clp.sel(clp_label=ca_doas_labels)).sum(
    dim="clp_label"
)

data = xr.Dataset(data_dict)

plot_dim = (1, 1)
myFRLcolors = [
    "tab:orange",
    "tab:grey",
    "k",
    "r",
    "b",
    "g",
    "m",
    "c",
    "y",
    "tab:brown",
    "tab:purple",
]
custom_cycler = cycler(color=myFRLcolors)
fig, ax = plt.subplots(*plot_dim, figsize=(8, 4))
#
wl = 510
ax.set_prop_cycle(custom_cycler)
for key in data.data_vars.keys():
    # Shift by IRF
    irf_location = extract_irf_location(result.data["4TT"], wl)
    shift_time_axis_by_irf_location(
        data[key].sel(spectral=wl, method="nearest"), irf_location=irf_location
    ).plot(x="time", ax=ax, label=key)
    ax.set_xscale("symlog", linthresh=0.3)


# for ax, wl in zip(axes.flatten(), wls):
#     for key in data.data_vars.keys():
#         data[key].sel(spectral=wl, method="nearest").plot(x="time", ax=ax, label=key)

ax.legend(bbox_to_anchor=(0.02, 0.95), loc="upper left")
ax.set_xlabel("Time (ps)")
ax.set_ylabel("ΔA(mOD)")
ax_title = ax.get_title()
ax.set_title(rf"{ax_title.replace('spectral', 'Wavelength')}$\,$nm")


fig.tight_layout()

## Residual analysis of the corrected data

In [ ]:
def plot_residual_and_svd(result_dataset):
    fig, axes = plt.subplots(1, 3, figsize=(10, 2))
    plot_residual(result_dataset, axes[0])
    axes[0].set_title("Residual Matrix")
    axes[0].get_legend().remove()
    axes[0].set_ylabel("Wavelength (nm)")
    axes[0].set_xlabel("Time (ps)")
    plot_lsv_residual(result_dataset, axes[1], indices=[0])
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")
    axes[1].set_title("residual 1st LSV")
    axes[1].set_xlabel("Time (ps)")
    plot_rsv_residual(result_dataset, axes[2], indices=[0])
    axes[2].set_xlabel("Wavelength (nm)")
    axes[2].set_title("residual 1st RSV")
    axes[2].get_legend().remove()
    axes[2].set_ylabel("")

    return fig, axes


fig, axes = plot_residual_and_svd(result_corr_refined.data["corr4TT"])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

## Overview of the estimated DOAS and phases of the corrected data

In [ ]:
from pyglotaran_extras import plot_doas
from pyglotaran_extras.plotting.style import ColorCode

fig, axes = plot_doas(
    result_corr_refined,
    damped_oscillation=["osc1", "osc2", "osc3", "osc4"],
    time_range=(-0.1, 0.6),
    spectral=550,
    figsize=(15, 4),
    normalize=False
    # oscillation_type="sin",
    # center_λ=550,
)

for vline_pos in [415, 460]:
    axes[1].axvline(vline_pos, color="r", linewidth=1)
    axes[2].axvline(vline_pos, color="r", linewidth=1)
for vline_pos in [526]:
    axes[1].axvline(vline_pos, color=ColorCode.green, linewidth=1)
    axes[2].axvline(vline_pos, color=ColorCode.green, linewidth=1)
for vline_pos in [393, 429, 479]:
    axes[1].axvline(vline_pos, color="b", linewidth=1)
    axes[2].axvline(vline_pos, color="b", linewidth=1)
axes[0].set_xlabel("Time (ps)")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[1].set_title("DOAS")
axes[0].annotate("A", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)